# Introducción a Pytorch

## Preparación

In [1]:
# !pip install torch torchvision torchaudio scikit-learn  # (si hace falta)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(0)  # reproducibilidad

# Datos sintéticos

In [ ]:
X, y = make_moons(n_samples=1000, noise=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)     # CrossEntropyLoss espera long
X_val   = torch.tensor(X_val, dtype=torch.float32)
y_val   = torch.tensor(y_val, dtype=torch.long)


In [ ]:
plt.scatter(X[:,0], X[:,1], c=y, s=10)
plt.title("Dos lunas (datos sintéticos)")
plt.show()


# Propagación hacia adelante (forward)

## 1. Forward “a mano” con tensores (ilustra el cálculo)

Definimos pesos aleatorios y aplicamos: X -> capa oculta -> ReLU -> salida (logits).

In [ ]:
# Dimensiones
in_features = 2
hidden = 16
out_features = 2  # dos clases

# Inicialización aleatoria (por defecto ~N(0,1) escalado según init de PyTorch si usas nn.Linear;
# aquí lo haremos explícito con torch.randn)
W1 = torch.randn(in_features, hidden, dtype=torch.float32) * 0.1
b1 = torch.zeros(hidden, dtype=torch.float32)

W2 = torch.randn(hidden, out_features, dtype=torch.float32) * 0.1
b2 = torch.zeros(out_features, dtype=torch.float32)

# Forward manual (sin autograd por ahora, solo para ver el paso hacia adelante)
def forward_manual(X):
    z1 = X @ W1 + b1          # capa lineal 1
    h1 = F.relu(z1)           # activación
    logits = h1 @ W2 + b2     # capa lineal 2
    return logits

logits0 = forward_manual(X_train)     # [N, 2]
print("Logits iniciales (primeras 5 filas):\n", logits0[:5])


Pérdida inicial

In [ ]:
loss_fn = nn.CrossEntropyLoss()
loss0 = loss_fn(logits0, y_train)
print("Pérdida inicial (sin entrenar):", float(loss0))

## 2. Forward usando un nn.Module (lo normal en PyTorch)

In [ ]:
class MLP(nn.Module):
    def __init__(self, in_features=2, hidden=16, out_features=2):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden)  # inicializa pesos aleatoriamente
        self.fc2 = nn.Linear(hidden, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # logits (sin softmax para usar CrossEntropyLoss)
        return x

net = MLP(in_features=2, hidden=16, out_features=2)
logits = net(X_train)           # forward
loss = loss_fn(logits, y_train) # pérdida inicial
print("Pérdida inicial (nn.Module):", float(loss))


# Propagación hacia atrás y optimización

El ciclo de entrenamiento en PyTorch siempre sigue estos pasos:

1. `optimizer.zero_grad()` — limpiar gradientes acumulados
2. `logits = net(X_batch)` — **forward**
3. `loss = loss_fn(logits, y_batch)` — calcular pérdida
4. `loss.backward()` — **backward** (autograd calcula ∂loss/∂θ)
5. `optimizer.step()` — **actualiza** parámetros con el optimizador

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)  # también puedes usar Adam

def accuracy(model, X, y):
    with torch.no_grad():
        pred = model(X).argmax(dim=1)
        return (pred == y).float().mean().item()

print("Acc (antes):", accuracy(net, X_val, y_val))

for epoch in range(200):
    optimizer.zero_grad()
    logits = net(X_train)
    loss = loss_fn(logits, y_train)
    loss.backward()          # calcula gradientes d(loss)/d(param)
    optimizer.step()         # actualiza pesos

    if (epoch+1) % 40 == 0:
        acc_tr = accuracy(net, X_train, y_train)
        acc_va = accuracy(net, X_val, y_val)
        print(f"Época {epoch+1:3d} | loss={loss.item():.4f} | acc_tr={acc_tr:.3f} | acc_val={acc_va:.3f}")

print("Acc (después):", accuracy(net, X_val, y_val))

# Frontera de decisión visual

In [ ]:
# Malla para visualizar la frontera de decisión
xx, yy = np.meshgrid(
    np.linspace(X[:,0].min()-0.5, X[:,0].max()+0.5, 300),
    np.linspace(X[:,1].min()-0.5, X[:,1].max()+0.5, 300)
)
grid = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)

with torch.no_grad():
    Z = net(grid).argmax(dim=1).reshape(xx.shape).numpy()

plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X_val[:,0], X_val[:,1], c=y_val, s=10)
plt.title("Frontera de decisión (modelo entrenado)")


# Control exlpícito de la inicialización

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0.0, std=0.1)
        nn.init.zeros_(m.bias)

net = MLP()
net.apply(init_weights)  # aplica tu init definida arriba


# Data loader

In [2]:
from torch.utils.data import TensorDataset, DataLoader

X, y = make_moons(n_samples=1000, noise=0.2, random_state=0)

# Control exlpícito de la inicialización
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0)


In [4]:
X_train.shape

(700, 2)

In [6]:
y_train.shape

(700,)

In [7]:
# Ejemplo de uso de Data loader de Pytorch

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_val = torch.FloatTensor(X_val)
y_val = torch.LongTensor(y_val)

In [9]:
X_train.shape

torch.Size([700, 2])

In [12]:
y_train.shape

torch.Size([700])

In [13]:
# Create datasets and dataloaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

In [18]:
batch_size = 200
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for batch_X, batch_y in train_loader:
    print(batch_X.shape, batch_y.shape)
    

torch.Size([200, 2]) torch.Size([200])
torch.Size([200, 2]) torch.Size([200])
torch.Size([200, 2]) torch.Size([200])
torch.Size([100, 2]) torch.Size([100])


In [32]:
itertrain = iter(train_loader)

In [33]:
xbatch, ybatch = next(itertrain)
print(xbatch.shape, ybatch.shape)

torch.Size([200, 2]) torch.Size([200])


In [35]:
# Definimos la arquitectura de la red

class MLP(nn.Module):
    def __init__(self, in_features=2, hidden=16, out_features=2):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden)  # inicializa pesos aleatoriamente
        self.fc2 = nn.Linear(hidden, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # logits (sin softmax para usar CrossEntropyLoss)
        return x

In [36]:
net = MLP(in_features=2, hidden=16, out_features=2)

# Definimos funciones de pérdida y optimizador
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9) 

In [39]:
for epoch in range(10):
    for batch_X, batch_y in train_loader:
        print(batch_X.shape, batch_y.shape)
        logits = net(batch_X)           # forward
        loss = loss_fn(logits, batch_y) # pérdida inicial
        print("Pérdida inicial (nn.Module):", float(loss))
        loss.backward()          # calcula gradientes d(loss)/d(param)
        optimizer.step()         # actualiza pesos
    

torch.Size([200, 2]) torch.Size([200])
Pérdida inicial (nn.Module): 8.128868103027344
torch.Size([200, 2]) torch.Size([200])
Pérdida inicial (nn.Module): 7.932872772216797
torch.Size([200, 2]) torch.Size([200])
Pérdida inicial (nn.Module): 6.966192245483398
torch.Size([100, 2]) torch.Size([100])
Pérdida inicial (nn.Module): 6.437458038330078
torch.Size([200, 2]) torch.Size([200])
Pérdida inicial (nn.Module): 6.032983779907227
torch.Size([200, 2]) torch.Size([200])
Pérdida inicial (nn.Module): 5.6277570724487305
torch.Size([200, 2]) torch.Size([200])
Pérdida inicial (nn.Module): 5.723283290863037
torch.Size([100, 2]) torch.Size([100])
Pérdida inicial (nn.Module): 5.160256862640381
torch.Size([200, 2]) torch.Size([200])
Pérdida inicial (nn.Module): 2.874570369720459
torch.Size([200, 2]) torch.Size([200])
Pérdida inicial (nn.Module): 1.5430777072906494
torch.Size([200, 2]) torch.Size([200])
Pérdida inicial (nn.Module): 0.8315367102622986
torch.Size([100, 2]) torch.Size([100])
Pérdida inic